# Датасет "Оставшееся количество циклов зарядки аккумулятора" 
Оригинальное название: *Battery Remaining Useful Life (RUL)*


## Клиент для удаленного подключения к серверу TensorFlow Serving

In [1]:
import requests
import pickle
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Загрузка сохраненных объектов MinMaxScaler:

In [26]:
with open('app/MinMaxScalers/mmsx.pkl', 'rb') as f:
    loaded_MMS_X = pickle.load(f)
    f.close()
    
with open('app/MinMaxScalers/mmsy.pkl', 'rb') as f:
    loaded_mms_y = pickle.load(f)
    f.close()

Загрузка проверочной части датасета:

In [3]:
df_X_test = pd.read_csv('app/test_data/Battery_RUL_X_test.csv', index_col=0)
df_y_test = pd.read_csv('app/test_data/Battery_RUL_y_test.csv', index_col=0)

In [4]:
df_X_test

,F1,F4,F7
11339,1511.75,3.593,8317.81
14156,2003.22,3.464,8897.94
7356,1006.34,3.690,8010.82
1669,1538.84,3.584,8407.30
7536,781.19,3.734,7349.25
...,...,...,...
5608,1966.53,3.466,9002.19
42,2217.28,3.379,9115.94
3144,1032.06,3.683,7343.72
3161,1019.38,3.684,7359.34


In [5]:
df_y_test

,F2,F3,F5,F6,RUL
11339,394.285714,3.864,2701.175000,3572.38,533
14156,662.400000,3.983,4393.113000,5336.31,964
7356,266.785714,3.804,1448.253000,2060.25,192
1669,410.742857,3.861,2834.360000,3716.36,492
7536,179.306667,3.749,929.975000,1412.38,12
...,...,...,...,...,...
5608,649.600000,3.985,4316.359000,5264.36,882
42,830.407000,4.045,5062.375000,6035.38,1069
3144,276.571429,3.810,1515.208143,2168.35,87
3161,272.043956,3.808,1470.812000,2096.31,70


Функция для получения предсказания с сервера TensorFlow Serving и его печати:

In [6]:
def get_prediction(X):
    
    """
    X должен быть в формате [ X0, X1, ... Xn ], где каждый элемент Xi = [ F1, F4, F7 ] и n >= 0
    
    """
        
    try:
        Xt = loaded_MMS_X.transform(X).tolist()
        data = '{"instances": ' + str(Xt) + '}'
    except:
        print('Ошибка: неверный тип X')
        return
    
    conn_error = False

    try:
        pred = requests.post(
            "http://localhost:8501/v1/models/Battery_RUL:predict", 
            data=data)
    except:
        conn_error = True

    if conn_error:
        print('Ошибка соединения с сервером.')
        return
    
    if pred.status_code == 200:
        
        if pred.json().get('predictions', -1) == -1:
            print('Неизвестная ошибка.')
            return
        else:
            print(
                '{:.2f} {:.3f} {:.2f} {:.2f} {:.2f}'.format(
                *loaded_mms_y.inverse_transform(pred.json()['predictions'])[0]) )

            return
    
    else:
    
        if pred.json().get('error', -1) == -1:
            print('Неизвестная ошибка.')
            return
        else:
            print( pred.json()['error'] )

Получаем предсказание с сервера:

In [31]:
get_prediction( [list(df_X_test.loc[42])] )

830.21 4.009 5028.00 6022.53 1068.01


Сравниваем с y_test:

In [8]:
print('{:.2f} {:.3f} {:.2f} {:.2f} {:.2f}'.format(*df_y_test.loc[42].tolist()))

830.41 4.045 5062.38 6035.38 1069.00
